In [1]:
import time
import pandas as pd
import boto3
import json
import configparser

In [2]:
config = configparser.ConfigParser()
config.read_file(open('creds.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

In [3]:
config.read_file(open('dwh.cfg'))

DB_IDENTIFIER          = config.get("CLUSTER","DB_IDENTIFIER")
DB_NAME                = config.get("CLUSTER","DB_NAME")
DB_USER                = config.get("CLUSTER","DB_USER")
DB_PASSWORD            = config.get("CLUSTER","DB_PASSWORD")
DB_PORT                = config.get("CLUSTER","DB_PORT")
HOST                   = config.get("CLUSTER","HOST")

IAM_ROLE_NAME          = config.get("IAM_ROLE", "IAM_ROLE_NAME")
ROLE_ARN               = config.get("IAM_ROLE", "ARN")

LOG_DATA               = config.get("S3", "LOG_DATA")
LOG_JSONPATH           = config.get("S3", "LOG_JSONPATH")
SONG_DATA              = config.get("S3", "SONG_DATA")

In [4]:
# create redshift client
redshift = boto3.client('redshift',
                        region_name='us-west-2', 
                        aws_access_key_id=KEY, 
                        aws_secret_access_key=SECRET
                       )

# create iam client
iam = boto3.client('iam',
                    region_name='us-west-2', 
                    aws_access_key_id=KEY, 
                    aws_secret_access_key=SECRET
                   )

In [6]:
# # delete cluster 
redshift.delete_cluster(ClusterIdentifier=DB_IDENTIFIER, 
                        SkipFinalClusterSnapshot=True)

In [7]:
# check if the cluster is available
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", 
                  "MasterUsername", "DBName", "Endpoint", 
                  "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

In [8]:
while True:
    try:
        myClusterProps = (redshift
                          .describe_clusters(ClusterIdentifier=DB_IDENTIFIER)
                          ['Clusters'][0])
        prettyRedshiftProps(myClusterProps)
        time.sleep(10)

    except Exception as e:
        print(e)
        break

An error occurred (ClusterNotFound) when calling the DescribeClusters operation: Cluster dwhcluster not found.


In [10]:
# delete the created resources
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)